In [1]:
# Goal: Generate content for music inspired cocktails
template = {
    "title": "", # get list of song names. find rhymes for each word. replace closest word to alcohol. 
    "description": "", # tracery grammar 
    "brewed_date": "", # date of song
    "style": "", # music genre
    "alcohol_content": "", # random num?
    "notes": "", # list of misc liqeurs
    "pairing": "", # find list of song names. find rhymes for each word, replace closest word to X food
    "serving_suggestion": "" # tracery grammar using words from lyrics (closest words to temperature etc.)
}

In [6]:
# Download data, shell script
# song lyrics
! wget -O data/lyrics.csv \
https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv 

--2017-09-24 10:08:05--  https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv
Resolving raw.githubusercontent.com... 151.101.164.133
Connecting to raw.githubusercontent.com|151.101.164.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7953541 (7.6M) [text/plain]
Saving to: ‘scratch_data/lyrics.csv’

scratch_data/lyrics 100%[===================>]   7.58M  --.-KB/s    in 0.1s    

2017-09-24 10:08:05 (51.7 MB/s) - ‘scratch_data/lyrics.csv’ saved [7953541/7953541]



In [122]:
# Create json of cocktail info
import requests
from bs4 import BeautifulSoup, Comment

data = {
    'cocktails': {},
    'source': 'http://www.cocktaildb.com'
}

def return_text(x):
    return x.get_text()

for i in range(4758):
    if (i % 10 == 0): # download every 10th entry to save time
        url = 'http://www.cocktaildb.com/recipe_detail?id={}'.format(i+1)
        raw_html = requests.get(url).text
        soup = BeautifulSoup(raw_html, 'html.parser')
        if (soup):
            cocktail = soup.find(id='wellTitle')
            if (cocktail):
                c_name = cocktail.find('h2').get_text()
                data['cocktails'][c_name] = {
                    'recipe': list(map(return_text, soup.find_all(class_='recipeMeasure'))),
                    'directions': list(map(return_text, soup.find_all(class_='recipeDirection')))
                }
                print(i)

import json
with open('data/cocktails.json', 'w') as outfile:
    json.dump(data, outfile)
    print('File written')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [124]:
#  Read lyrics names
import csv
lyrics_raw = csv.DictReader(open('data/lyrics.csv',  encoding='ISO-8859-1'))
lyrics = []
for row in lyrics_raw:
    # Keys: Rank, Song, Artist, Year, Lyrics, Source
    not_obscure = (int(row['Year']) >= 2010 and int(row['Rank']) < 50) or (int(row['Rank']) <= 5 and int(row['Year']) >= 1994)
    if (not_obscure) :
        lyrics.append({
            'rank': row['Rank'],
            'song': row['Song'],  
            'artist': row['Artist'],
            'year': row['Year'],
            'lyrics': row['Lyrics'],
            'source': row['Source']
        });


{'rank': '16', 'song': 'i knew you were trouble', 'artist': 'taylor swift', 'year': '2013', 'lyrics': ' once upon time a few mistakes ago i was in your sights you got me alone you found me you found me you found mei guess you didnt care and i guess i liked that and when i fell hard you took a step back without me without me without meand hes long gone when hes next to me and i realize the blame is on mecause i knew you were trouble when you walked in so shame on me now flew me to places id never been so you put me down oh i knew you were trouble when you walked in so shame on me now flew me to places id never been now im lying on the cold hard ground oh oh trouble trouble trouble oh oh trouble trouble troubleno apologies hell never see you cry pretend he doesnt know that hes the reason why youre drowning youre drowning youre drowningand i heard you moved on from whispers on the street a new notch in your belt is all ill ever be and now i see now i see now i see he was long gone when he

In [401]:
# NLP vector helpers
def vec(s):
    return nlp.vocab[s].vector

import numpy as np
from numpy import dot
from numpy.linalg import norm

# cosine similarity
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0
    
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine(vec_to_check, vec(x)),
                  reverse=True)[:n]


def word_count(token_list):
    count = {} 
    for token in token_list:
        text = str(token.text).strip()
        if (len(text) > 0):
            if (text not in count.keys()):
                count[text] = 0

            count[text] += 1
    return count

def addv(coord1, coord2):
    return [c1 + c2 for c1, c2 in zip(coord1, coord2)]

In [615]:
# Selecting random data
import random

import json
drink_data = {};
with open('data/cocktails.json') as d:
    data = json.load(d)
    d.close()
    drink_data = data

song = random.choice(lyrics)
d = random.choice(list(drink_data['cocktails'].items()))
drink = {
    'name': d[0],
    'directions': d[1]['directions'],
    'recipe': d[1]['recipe']   
}
print(song['song'], song['artist'], drink['name'])

from __future__ import unicode_literals
import spacy

def humanise_name(n):
    return n.lower().strip().title()

def get_drink_title(s, d):
    case = random.randint(0, 1)
    if (len(s['song'].split()) > 1 and case == 1):  # replace alcohol with artist name
        nlp = spacy.load('en')
        song_title_doc = nlp(s['song'])
        drink_doc = nlp(d['name'])
        case2 = random.randint(0, 1)
        name = []
        replaced = False
        for w in song_title_doc:
            print(w.pos_)
            if ((w.pos_ == 'NOUN' or w.pos_ == 'ADJ' or w.pos_ == 'PRON') and replaced == False):
                drink_tokens = list(set([z.text for z in drink_doc if z.is_alpha]))
                neighbours = spacy_closest(drink_tokens, vec('liquour'))
                if (len(neighbours) >= 1):
                    alcohol_name = neighbours[0]
                    name.append(alcohol_name)
                    replaced = True
            else:
                name.append(w.text)
        return humanise_name(' '.join(name))
    else: # artist's #DRINK#
        apostrophe = '\'' if (s['artist'][-1].lower() == 's') else '\'s'
        return '{}{} {}'.format(humanise_name(s['artist']), apostrophe, d['name'])

   
    

def get_drink(s, d):
    content = {
        'title': get_drink_title(s, d),
        'description': '',
        'brewed_date': '',
        'style': '',
        'alcohol_content': '',
        'notes': '',
        'pairing': '',
        'serving_suggestion': '',
        'image': ''
    }
    return content

get_drink(song, drink)



# nlp = spacy.load('en')
# lyric_doc = nlp(song['lyrics'])
# lyric_tokens = list(set([w.text for w in lyric_doc if w.is_alpha]))

# artist_doc = nlp(song['artist'])
# artist_tokens = list(set([w.text for w in artist_doc if w.is_alpha]))

# song_doc = nlp(song['song'])
# song_tokens = list(set([w.text for w in song_doc if w.is_alpha]))


# drink_name_doc = nlp(drink[0])
# drink_tokens = list(set([w.text for w in drink_name_doc if w.is_alpha]))

# for word in drink_name_doc:
#     print(get_pos(word))
# print(drink[0], song['song'], song['artist'])

# # spacy_closest(song_tokens, vec('alcohol'))
# spacy_closest(lyric_tokens, addv(vec("Palmetto"), vec("water")))
# import tracery
# from tracery.modifiers import base_english



# drink

heartbreak hotel whitney houston featuring faith evans and kelly price Queen Bee
NOUN
NOUN


{'alcohol_content': '',
 'brewed_date': '',
 'description': '',
 'image': '',
 'notes': '',
 'pairing': '',
 'serving_suggestion': '',
 'style': '',
 'title': 'Queen Hotel'}

In [469]:
random.randint(0,3)

1